In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver
import operator
from langchain_core.messages import  AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults




In [48]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

system_prompt = """ You are an event manager planning a seminar about various software engineering topics happening at Hunt Valley, Maryland. 
The user will ask you to inquire about topics of the seminar.
The user will ask you to register for certain sessions, you need to ask them the required information and register for the topics.
Always confirm with user before registering them for any topic.
Allow the user to modify or cancel their registration.
The user may inquire about their existing registrations, you need to provide them with the list of topics they are registered for.
If certain topic that is not available as session, then note down the feedback from the user and thank them. But do not register for any topic that you are not provided with.
Keep the chat limited to only the seminar topics and registration.
If any of the tools are not available, then you need to inform the user about it and tell that this will be added in the next version.
Do not engage into any emotional or intimate conversations with the user, politely decline if the user starts such a topic.
At the end of a successful registration offer the user to book nearby hotels and suggest food options. For this you can use the TavilySearchResults tool.
You can also provide information about tourist places and current weather using TavilySearchResults tool.
"""

welcome_prompt = """Welcome to the Technology seminar of North Maryland Area. How can I assist you today? When you are done chatting, please write bye to end"""


In [49]:
class RegistrationState(TypedDict):
    messages: Annotated[list, add_messages]
    messages: Annotated[list[AnyMessage], operator.add]

memory = MemorySaver()


### Define LLM node and other nodes for the State Graph

In [50]:
class RegistrationAgent:
    def __init__(self,model,tools,checkpointer, system=""):
        # Save the system message as a class level attribute
        self.system = system

        # Initialize the state graph that will have one LLM node, One Tool node and one Action Edge
        graph = StateGraph(RegistrationState)
        # Start Node
        graph.add_node("llm",self.call_llm)
        
        # Action Node that is available as tool
        graph.add_node("action",self.action_node)

        # Decision Edge to decide to use action node
        # First parameter is the node from which this edge is coming from 
        # Second parameter is the function that let's langraph explore tools
        # Third parameter is available nodes after the decision either action node or END node
        graph.add_conditional_edges("llm",self.action_edge_decision,{True: "action", False: END})

        # Create Another edge to loop back to LLM node from action node
        graph.add_edge("action","llm")

        # Define what node the graph should start, in this case the llm
        graph.set_entry_point("llm")

        # Once setup done compile the graph and Save the graph at the class level
        self.graph = graph.compile(checkpointer=checkpointer)

        # Create a dictionary of available tools sent to the constructor
        self.tools = {tool.name: tool for tool in tools}

        # Bind tools to model so that LLM can search for tools
        self.model = model.bind_tools(tools)


    # Define function for call llm node
    def call_llm(self, state: RegistrationState):
        # get the messages saved in the Agent state object
        messages = state["messages"]
        # If system message is not blank, append that to the beginning of the messages
        if self.system:
            system_message= [SystemMessage(content=self.system)]
            messages =  system_message + messages
        # Call the model with the messages, it should return response as a single message
        resp = self.model.invoke(messages)
        print(f"Response from LLM -> {resp}")
        # Return the response message as a list, that will be appended to the existing messages due to operator.add annotation at class level
        return {"messages": [resp]}
    
    # Define function for call action node
    def action_node(self, state: RegistrationState):
        # get the last message from the Agent State, since the last message is the response from LLM that suggests to use the tool
        # tool calls attribute is expected which has the name of the tool to be called
        referred_tools_list = state["messages"][-1].tool_calls
        results = []

        # Tool calls can be multiple tools, so iterate over them
        for tool in referred_tools_list:
            print(f"Tool to be called -> {tool}")
            # invoke tool call by finding the name and the arguments as suggested by LLM
            result = self.tools[tool["name"]].invoke(tool["args"])
            # Append the result to the results list
            results.append(ToolMessage(tool_call_id=tool["id"], name=tool["name"], content=str(result)))
            
        print(f"Finished tool call ...")
        # returns results and add to the messages list at class level
        return {"messages": results}
    
    # Define the actiton edge function that decides whether to look for tool or not
    # If the last message in the message list has tool_calls attribute, then return True, else False
    def action_edge_decision(self, state: RegistrationState):
        result = state["messages"][-1]
        return  len(result.tool_calls) > 0



### Prepare LLM with Agent Graph and Memory configuration

In [51]:
tool = TavilySearchResults(max_results=3)
reg_agent = RegistrationAgent(llm,[tool],memory, system_prompt)
# Add a thread id to make the conversation persistent
thread_id = {"configurable": {"thread_id":"1"}}


### Invoke Model
- Apply recursive limit so that the chat will end after these many calls

In [ ]:
user_input = [HumanMessage(content=input("User : "))]
events = reg_agent.graph.stream(
    {"messages": user_input},
    thread_id,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()